In [24]:
import sys
from pathlib import Path

# 1. Configuration des chemins
# On remonte d'un cran pour atteindre la racine du projet
ROOT_DIR = Path("..").resolve()
sys.path.append(str(ROOT_DIR))

# 2. Rechargement automatique (Autoreload)
%load_ext autoreload
%autoreload 2

# 3. Import du module
from src.data_utils import MultilingualIndexer

print(f"Racine du projet configurée : {ROOT_DIR}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Racine du projet configurée : C:\Users\benic\Documents\Projet_DL_Translation


In [25]:
# Instanciation de l'indexeur
indexer = MultilingualIndexer(root_dir=ROOT_DIR)

In [ ]:
# Liste des archives à traiter
archives = [
    "mtedx_fr.tgz", 
    "mtedx_fr-en.tgz", 
    "mtedx_fr-es.tgz", 
    "mtedx_fr-pt.tgz"
]

print("--- Extraction des métadonnées ---")
# Cette étape est rapide (pas d'extraction audio lourde)
indexer.extract_metadata_files(archives)
print("--- Extraction terminée ---")

--- Début de l'extraction des métadonnées ---
Extraction des métadonnées de mtedx_fr.tgz...
Extraction des métadonnées de mtedx_fr-en.tgz...
Extraction des métadonnées de mtedx_fr-es.tgz...
Extraction des métadonnées de mtedx_fr-pt.tgz...
--- Extraction terminée ---


In [26]:
try:
    # Création du DataFrame fusionné (FR + Traductions)
    # C'est ici que la magie du Lazy Loading et des chemins wav/vtt opère
    df_master = indexer.create_master_index(split='train')
    
    print("\n✅ Index Maître généré avec succès en mémoire.")

except Exception as e:
    print(f"❌ Erreur critique lors de la création de l'index : {e}")
    # Astuce : On raise l'erreur pour arrêter l'exécution des cellules suivantes si on fait 'Run All'
    raise e

Construction de l'Index Maitre (Mode Aligne ID) pour : train
Total Segments FR : 116045
Succes ! CSV Maitre genere : C:\Users\benic\Documents\Projet_DL_Translation\data\processed\master_metadata.csv (116045 lignes)

✅ Index Maître généré avec succès en mémoire.


In [27]:
if 'df_master' in locals():
    # Affichage des statistiques de couverture
    indexer.calculate_coverage_stats(df_master)
    
    # Aperçu des 5 premières lignes
    print("\n--- Aperçu des données ---")
    import pandas as pd
    pd.set_option('display.max_colwidth', None)
    display(df_master.head())
    pd.reset_option('display.max_colwidth')


--- Statistiques de Couverture (N=116045) ---
Traduction EN: 30171 (26.00%)
Traduction ES: 20826 (17.95%)
Traduction PT: 13286 (11.45%)

--- Aperçu des données ---


,segment_id,video_id,start_time,end_time,wav_path,vtt_path,text_fr,text_en,text_es,text_pt
0,0-DmpK7Di0c_0000,0-DmpK7Di0c,8.57,9.45,data\temp\fr-fr\data\train\wav\0-DmpK7Di0c.wav,data\temp\fr-fr\data\train\vtt\0-DmpK7Di0c.vtt,"Mesdames et messieurs, bonjour.",NaN,"Señoras y señores, buenos días.","Senhoras e senhores, bom dia."
1,0-DmpK7Di0c_0001,0-DmpK7Di0c,9.91,12.93,data\temp\fr-fr\data\train\wav\0-DmpK7Di0c.wav,data\temp\fr-fr\data\train\vtt\0-DmpK7Di0c.vtt,"Je vais partager avec vous, en français, ma passion : l'aéronautique.",NaN,"Voy a compartir mi pasión con Uds., en francés: la aeronáutica.","Vou falar-vos, em francês, da minha paixão: a aeronáutica."
2,0-DmpK7Di0c_0002,0-DmpK7Di0c,15.11,25.47,data\temp\fr-fr\data\train\wav\0-DmpK7Di0c.wav,data\temp\fr-fr\data\train\vtt\0-DmpK7Di0c.vtt,"Tout ceci commence par une anecdote troublante qui m'est arrivée il y a quelques années, lorsqu'un ami me contacta, étant passionné d'aviation et habitant Toulouse.",NaN,"Todo comienza con una anécdota inquietante que me ocurrió hace unos años cuando me contactó un amigo, un apasionado de la aviación y que vivía en Toulouse","Tudo isto começa por uma história desconcertante que me aconteceu há uns anos, quando um amigo me contactou. Era um apaixonado da aviação e morava em Toulouse."
3,0-DmpK7Di0c_0003,0-DmpK7Di0c,26.04,30.76,data\temp\fr-fr\data\train\wav\0-DmpK7Di0c.wav,data\temp\fr-fr\data\train\vtt\0-DmpK7Di0c.vtt,"Il avait trouvé des pièces d'avion dans la terre au sud de Toulouse, et il me demanda de les identifier.",NaN,Había encontrado piezas de avión en la tierra al sur de Toulouse y quería que las identificara.,Tinha encontrado peças de avião numa terra a sul de Toulouse e pedia-me para as identificar.
4,0-DmpK7Di0c_0004,0-DmpK7Di0c,31.92,36.17,data\temp\fr-fr\data\train\wav\0-DmpK7Di0c.wav,data\temp\fr-fr\data\train\vtt\0-DmpK7Di0c.vtt,"Donc rapidement, je mis des photos de ces pièces sur un site internet spécialisé.",NaN,Así que puse rápidamente fotos de las piezas en una página web especializada.,"Rapidamente, coloquei fotos dessas peças numa página especializada da Internet."


In [28]:
if 'df_master' in locals():
    # Crée les fichiers propres pour Whisper (ASR) et MarianMT (NMT)
    # Sépare les données monolingues et bilingues
    indexer.create_specialized_datasets(df_master)
else:
    print("Veuillez d'abord générer le master index (Cellule 4).")


--- Generation des Datasets Specialises ---
âœ… [ASR] Dataset FR genere : C:\Users\benic\Documents\Projet_DL_Translation\data\processed\train_asr_fr.csv (116045 segments)
   -> Pour Fine-tuning Whisper
âœ… [NMT] Dataset Parallele FR-EN genere : C:\Users\benic\Documents\Projet_DL_Translation\data\processed\train_nmt_fr_en.csv (30171 paires)
âœ… [NMT] Dataset Parallele FR-ES genere : C:\Users\benic\Documents\Projet_DL_Translation\data\processed\train_nmt_fr_es.csv (20826 paires)
âœ… [NMT] Dataset Parallele FR-PT genere : C:\Users\benic\Documents\Projet_DL_Translation\data\processed\train_nmt_fr_pt.csv (13286 paires)
